# Sample Authentication Methods

## Prerequisites

* Running Vault Server at localhost:8200 - see [Start Vault Server](./100-Setup-Vault.ipynb)

### Set Up Environment Variables

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"
# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history
# Set dir env vars. 
MAIN_DIR=$(pwd)
WORK_DIR=config/vault

Customize the values below if you want Vault to start with different parameters

In [ ]:
#Set Env. This assumes Dev Mode.
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:8200
MAIN_DIR=$(pwd)
WORK_DIR=config/vault

## LDAP

### Start LDAP Server container

This starts a Docker container with an LDAP server that can be configured as an authentication source.

In [ ]:
docker run \
  --name ldap-service \
  --env LDAP_ORGANISATION="engineers" \
  --env LDAP_DOMAIN="example.org" \
  --env LDAP_ADMIN_PASSWORD="admin" \
  -p 389:389 \
  -p 636:636 \
  --detach \
  --rm \
  osixia/openldap:latest

### Add LDAP Users

Install ldap utilities.

In [ ]:
brew install openldap

Create `dev` group with two members, `alice` and `bob`.

In [ ]:
# docker exec ldap-service \
ldapadd -cxD "cn=admin,dc=example,dc=org" -w admin <<EOF
dn: ou=groups,dc=example,dc=org
objectClass: organizationalunit
objectClass: top
ou: groups
description: groups of users

dn: ou=users,dc=example,dc=org
objectClass: organizationalunit
objectClass: top
ou: users
description: users

dn: cn=dev,ou=groups,dc=example,dc=org
objectClass: groupofnames
objectClass: top
description: testing group for dev
cn: dev
member: cn=alice,ou=users,dc=example,dc=org
member: cn=bob,ou=users,dc=example,dc=org


dn: cn=alice,ou=users,dc=example,dc=org
objectClass: person
objectClass: top
cn: alice
sn: lee
memberOf: cn=dev,ou=groups,dc=example,dc=org
userPassword: password123

dn: cn=bob,ou=users,dc=example,dc=org
objectClass: person
objectClass: top
cn: bob
sn: villa
memberOf: cn=dev,ou=groups,dc=example,dc=org
userPassword: password123

dn: cn=ansible,ou=users,dc=example,dc=org
objectClass: person
objectClass: top
cn: ansible
sn: ansible
userPassword: password123

EOF

* `-c` continous operation mode (do not stop on errors)
* `-x` simple authentication
* `-W` prompt for bind password
* `-D` binddn 

### Verification - Search ldap (test)

In [ ]:
docker exec ldap-service \
  ldapsearch -x -H ldap://host.docker.internal \
  -b cn=alice,ou=users,dc=example,dc=org \
  -D "cn=admin,dc=example,dc=org" -w admin

Sample Output

### Setup Vault LDAP Auth Method

In [ ]:
vault auth enable ldap

In [ ]:
vault write auth/ldap/config \
    url=ldap://host.docker.internal \
    userdn="ou=users,dc=example,dc=org" \
    groupdn="ou=groups,dc=example,dc=org" \
    binddn="cn=admin,dc=example,dc=org" \
    bindpass=admin \
    token_policies=""

### Configure Web Admin frontend to LDAP

In [ ]:
# docker rm phpldapadmin-service
docker run --rm -d \
    --name phpldapadmin-service --hostname phpldapadmin-service \
    -p 6443:443 \
    --link ldap-service:ldap-host \
    --env PHPLDAPADMIN_LDAP_HOSTS=ldap-host \
    osixia/phpldapadmin:latest

**Information about the containers**

In [ ]:
PHPLDAP_IP=$(docker inspect -f "{{ .NetworkSettings.IPAddress }}" phpldapadmin-service)
LDAP_IP=$(docker inspect -f "{{ .NetworkSettings.IPAddress }}" ldap-service)
echo "open browser to https://localhost:6443"
echo "phpweb IP is $PHPLDAP_IP; ldap IP is $LDAP_IP"
echo "Login DN: cn=admin,dc=example,dc=org"
echo "Password: admin"
open https://localhost:6443

### Test Login to Vault using LDAP credential

In [ ]:
VAULT_TOKEN="" vault login -method=ldap username=alice password=password123

### Cleanup - LDAP

In [ ]:
vault auth disable ldap

In [ ]:
docker stop ldap-service

In [ ]:
docker stop phpldapadmin-service

## UserPass

In [ ]:
#Set Env
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:8200

### Enable `userpass` auth engine.

In [ ]:
vault auth enable userpass

### Configure username/password

Create users. Set password and policies as well.

In [ ]:
vault write auth/userpass/users/alice password="passw0rd" policies="default",""
vault write auth/userpass/users/bob password="passw0rd" policies="default"
vault write auth/userpass/users/ansible password="passw0rd" policies="default"

## Approle

In [ ]:
#Set Env
export VAULT_TOKEN=root

### Enable auth engine - approle

In [ ]:
vault auth enable approle

### Configure auth engine - approle

In [ ]:
vault write -force auth/approle/role/vault-agent

### Configure auth engine role

In [ ]:
VAULT_ROLE_ID=$(vault read -field role_id auth/approle/role/vault-agent/role-id) \
  && echo $VAULT_ROLE_ID

In [ ]:
VAULT_SECRET_ID=$(vault write -f -format=json auth/approle/role/vault-agent/secret-id | jq -r .data.secret_id) \
  && echo ${VAULT_SECRET_ID}

### Verify auth engine - approle

In [ ]:
APP_TOKEN=$(vault write -field token auth/approle/login \
  role_id=${VAULT_ROLE_ID} \
  secret_id=${VAULT_SECRET_ID}) \
  && echo $APP_TOKEN

In [ ]:
VAULT_TOKEN=${APP_TOKEN} vault token lookup

## Clean Up

If you no longer need the auth engines, you can disable them.

In [ ]:
export VAULT_TOKEN=root

In [ ]:
vault auth disable approle

In [ ]:
vault auth disable userpass

In [ ]:
vault auth disable ldap